In [1]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.buffer_window import ConversationBufferWindowMemory
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from functools import reduce
import os

os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'https://openai-nois-intern.openai.azure.com/'
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "search-service01"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "public-data-index-alt"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9"

retriever_public = AzureCognitiveSearchRetriever(content_key='content')
print(retriever_public)

# os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "public-index-ver2"
# retriever_private = AzureCognitiveSearchRetriever(content_key='content')
# print(retriever_private)

service_name='search-service01' index_name='public-data-index-alt' api_key='73Swa5YqUR5IRMwUIqOH6ww2YBm3SveLv7rDmZVXtIAzSeBjEQe9' api_version='2020-06-30' aiosession=None content_key='content'


In [2]:
# Load up your LLM
llm = AzureChatOpenAI(
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='test-1',
    openai_api_key=os.getenv('OPENAI_API_KEY'),
    openai_api_type="azure"
)

output_parser = CommaSeparatedListOutputParser()
history = []

def get_history_as_txt():
    txt = ""
    for i in history:
        txt += f"\n<|im_start|>user\n{i['user']}\n<|im_end|>\n"
        txt += f"<|im_start|>assistant\n{i['AI']}\n<|im_end|>"
        
    return txt

def add_to_history(user_msg, ai_msg):
    history.append({'user': user_msg, 'AI': ai_msg})

In [3]:
print(llm)

verbose=False callbacks=None callback_manager=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.7 model_kwargs={} openai_api_key='400568d9a16740b88aff437480544a39' openai_api_base='https://openai-nois-intern.openai.azure.com/' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None deployment_name='test-1' openai_api_type='azure' openai_api_version='2023-03-15-preview'


In [4]:
keyword_templ = """<|im_start|>system
Given a sentence and a conversation about the companies New Ocean and NOIS, assistant will extract keywords based on the conversation and the sentence and translate them to Vietnamese if they're in English and vice versa.
Your output will be on one line, in both languages if possible and separated by commas. Assume the context surrounds the mentioned companies.
If given a sentence that is different in context compared to the conversation, do not return anything. Do not duplicate keywords.

EXAMPLE
Input: Ai chưa đóng tiền nước?
Output: tiền nước, water bill
Input: Was Pepsico a customer of New Ocean?
Output: Pepsico
Input: What is FASF?
Output: FASF
<|im_end|>
{context}
<|im_start|>user
Input: {question}
<|im_end|>
<|im_start|>assistant
Output:"""

query_prompt_template = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by querying relevant company documents.
Generate a search query based on the conversation and the new question. 

Chat History:{chat_history}

Question:
{question}

Search query:
"""

template = """<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
Sources:
{summaries}
<|im_end|>

{context}
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

templ = PromptTemplate(
    input_variables = ['summaries', 'question', 'context'],
    template = template
)

In [5]:
qaChain = load_qa_with_sources_chain(llm=llm, chain_type="stuff", prompt=templ)
keywordChain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(keyword_templ))

In [6]:
def get_docs_using_keyword_string(keywords, ret):
    return ret.get_relevant_documents(keywords)[:5]

In [10]:
query = "What are NOIS's future goals?"
keywords = keywordChain({'context': get_history_as_txt(), 'question': query})

doc = get_docs_using_keyword_string(keywords['text'], retriever_public)

num = 1
for x in doc:
    x.metadata['source'] = f"doc-{num}"

    num += 1

result = qaChain({'input_documents': doc, 'question': query, 'context': get_history_as_txt()}, return_only_outputs=False)
    
add_to_history(query, result['output_text'])

In [11]:
print(keywords['text'])

NOIS's future goals, mục tiêu tương lai của NOIS


In [12]:
result['output_text']

"According to source doc-2, NOIS's future goals include expanding beyond the company's profit targets, providing high-quality goods and services at affordable costs through intelligent production, and optimizing interactions between humans and machines in order to increase productivity and efficiency."

In [13]:
print(doc)

[Document(page_content='� liệu được sử dụng bởi toàn thể các thành viên trong công ty, nó cho phép nhân viên từ tất cả phòng ban khác nhau sử dụng nguồn cơ sở dữ liệu để phân tích từ đó đưa ra những nhận định, dự đoán xu hướng hoạt động tương lai và lập kế hoạch theo đuổi các mục tiêu kinh doanh.\r\nVAI TRÒ VÀ TẦM QUAN TRỌNG CỦA PHÂN TÍCH DỮ LIỆU TRONG THỜI KÌ CÔNG NGHIỆP 4.0\r\nKhái niệm dữ liệu lớn\n', metadata={'@search.score': 15.28319, 'metadata_storage_path': 'aHR0cHM6Ly9hY3NjaGF0Ym90bm9pc2ludGVybi5ibG9iLmNvcmUud2luZG93cy5uZXQvcHVibGljLWRhdGEtYWx0LzYwNi0zLnR4dA2', 'source': 'doc-1'}), Document(page_content='��t và chất lượng dự kiến sẽ vươn xa hơn mục tiêu là đáp ứng lợi nhuận của công ty. Trong tương lai sản xuất thông minh sẽ làm phong phú hơn cuộc sống của người tiêu dùng bằng cách cung cấp những hàng hóa, dịch vụ chất lượng cao với chi phí phải chăng.\r\nXem thêm: tiềm năng của Deep Learning trong tương lai\r\nThông tin công nghệ\r\nLỢI ÍCH VÀ NHỮNG LƯU Ý KHI ĐẦU TƯ TỰ ĐỘNG H

In [45]:
get_history_as_txt()

'\n<|im_start|>user\nNOIS là gì?\n<|im_end|>\n<|im_start|>assistant\nNOIS là viết tắt của New Ocean Information System, một đơn vị cung cấp giải pháp để hỗ trợ quản lý kinh doanh tối ưu. NOIS đã xác định vị trí của mình trong việc cung cấp các gói giải pháp để hỗ trợ quản lý kinh doanh tối ưu cho hơn 70 đối tác, bao gồm các thương hiệu nổi tiếng như Vinamilk, Dutch Lady, Amway, Nestle, P&G, HTL và nhiều hơn nữa. NOIS cũng nổi tiếng với việc luôn nắm bắt nhanh chóng các xu hướng công nghệ mới để đưa sự phát triển của công nghệ và chuyển đổi số vào các lĩnh vực sản xuất một cách thông minh, đúng nơi, đúng việc.\n<|im_end|>\n<|im_start|>user\nCông ty có tuyển vị trí data analysts không?\n<|im_end|>\n<|im_start|>assistant\nThe sources do not provide information on whether or not New Ocean or NOIS is currently hiring for data analyst positions. However, it is mentioned that data analysts are part of the data team at NOIS.\n<|im_end|>\n<|im_start|>user\nWho are the technical managers?\n<|im_

In [31]:
history = []

In [24]:
doc_txt = ""
for x in doc:
    doc_txt += x.page_content
    
context = get_history_as_txt()
    
msg = f"""<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
Sources:
{doc_txt}
<|im_end|>

{context}
"""

print(msg)
print(llm.get_num_tokens(msg))
print(f"Length of chat history: {llm.get_num_tokens(get_history_as_txt())}")

<|im_start|>system
Assistant helps the company employees and users with their questions about the companies New Ocean and NOIS. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
If the user asks any questions not related to New Ocean or NOIS, apologize and request another question from the user. If the user greets you, respond accordingly.
Sources:
Data Analytics, Data Engineering  & Machine Learning Portfolio
HomeData Analytics, Data Engineering  & Machine Learning Portfolio
// our portfolio
Data Analytics, Data Engineering  & Machine Learning

Data Analytics, Data Engineering  & Machine Learning Portfolio
HomeData Analytics, Data Engineering  & Machine Learning Portfolio
// our portfolio
Data Analytics, Data Engineering  & Machine Learning

APPLICATION OF

In [29]:
# Define query tool for agent
class DocumentQueryTool(BaseTool):
    name = "Document query tool"
    description = "use this tool when you need to get documents for questions related to New Ocean and NOIS"
    
    def _run(self, query: str):
        doc = retriever_public.get_relevant_documents(query)[:3]
        content = reduce(lambda x, y: x + y.page_content, doc, doc[0].page_content)
        
        return content
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

        
prefix = """You are an intelligent assistant helping New Ocean and NOIS employees as well as ordinary users with company related questions. You will be provided with keywords that you can use for querying documents.
If you cannot answer using the queried documents, say that you don't know.
You have access to the following tools:"""
suffix = """Begin!"

Keywords: {keywords}
Question: {input}
{agent_scratchpad}"""

tools = [DocumentQueryTool()]

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["keywords", "input", "agent_scratchpad"],
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [30]:
query = "Was Pepsico a customer of New Ocean?"
keywords = keywordChain({'question': query, 'context': get_history_as_txt()})['text']
print(keywords)

agent_chain.run(input=query, keywords=keywords)

Pepsico, New Ocean


> Entering new  chain...
Thought: I need to search for documents that contain information about Pepsico and New Ocean.
Action: Document query tool
Action Input: "Pepsico New Ocean customer"
Observation: NOIS Our Story
OUR STORY
Your Satisfaction Our Success
“
”
Our slogan “Your satisfaction, Our Success” highlights our
dedication to meeting the needs of our clients.
New Ocean IS
recognizes that satisfied customers are crucial to the success
of our business, not just in terms of repeat business but also
through positive word-of-mouth referrals. By prioritizing
customer satisfaction.
New Ocean IS
is able to cultivate a loyal customer base and establish
itself as a reputable and dependable company in the global
marketplace. Through this customer-centric approach.
New Ocean IS
is committed to achieving long-term success and growth by
delivering exceptional service and fostering enduring customer
relationships.
We’ve turned 13
Over the past 13 years,
New Ocean IS
has be

"I don't know if Pepsico was a customer of New Ocean."